# Climate Profiles 2

### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [1]:
from typing import Tuple
from typing import Any, Dict

import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm  # Progress bar

import climakitae as ck
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv, retrieve_profile_data, _filter_by_ssp
from climakitae.util.warming_levels import get_gwl_at_year
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)

import warnings
warnings.filterwarnings("ignore")

# import functions needed for testing
from climakitae.core.constants import UNSET
from climakitae.core.paths import VARIABLE_DESCRIPTIONS_CSV_PATH
from climakitae.explore.typical_meteorological_year import is_HadISD
from climakitae.explore.standard_year_profile import (
    _get_clean_standardyr_filename,
    _check_stations,
    _check_cached_area,
    _check_lat_lon,
    match_str_to_wl,
)
from climakitae.util.utils import read_csv_file
from climakitae.util.warming_levels import get_gwl_at_year

In [2]:
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_levels = [1.5]
no_delta = True

# warming level window size
valid_window = 5
invalid_window_1 = 2
invalid_window_2 = 5.5

# station name options
station_list = [
    "Sacramento Executive Airport (KSAC)",
    "Santa Barbara Municipal Airport (KSBA)",
]
station_name = ["Sacramento Executive Airport (KSAC)"]
custom_name = ["Custom Station Name"]
mixed_list = [
    "Custom Station Name",
    "Santa Barbara Municipal Airport (KSBA)",
]
custom_list = [
    "Custom Name 1",
    "Custom Name 2",
]

# lat lon
# latitude = 34.4041
# longitude = -121.5160 # no data found for these lat and lon bounds!
latitude = 34.19966
longitude = -118.36543  # no data found for these lat and lon bounds!

# cached area
area_name = "Los Angeles County"

### Time-based file modification

Valid inputs to test
- "approach" = "Time" with a "centered_year" between 2015 and 20199
- no "warming_level" input

Invalid inputs to test
1. "approach" = "Time" with a "centered_year" between 2015 and 2099, with "warming_level" input
2. "approach" = "Time" and no "centered_year" provided
3. "centered_year" outside of 2015-2099
4. "approach" that is not "Time" or "Warming Level"

profile_selections = {
    "variable": variable,
    "resolution": "9 km",
    "q": qtile,
    #"warming_level": [1.5],
    "units": units,
    "no_delta": no_delta,
    # approach
    "approach": "Time",
    "centered_year": 2016,
    "time_profile_scenario": "SSP 3-7.0",
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    "latitude": (
        latitude - 0.02,
        latitude + 0.02,
    ),  # uncomment for a using a custom coordinate location
    "longitude": (
        longitude - 0.02,
        longitude + 0.02,
    ),  # uncomment for a custom coordinate location
    # "cached_area": area_name, # uncomment for a cached area
}

results in:

No data found in bounds. Data variable: t2
Skipping spatial subsetting.

In [8]:
profile_selections = {
    "variable": "Precipitation (total)",
    "resolution": "9 km",
    "q": qtile,
    # "warming_level": [1.5],
    "units": units,
    "no_delta": no_delta,
    # approach
    "approach": "Time",
    "centered_year": 2016,
    "time_profile_scenario": "SSP 3-7.0",
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [9]:
profile = get_climate_profile(**profile_selections)

📊 Retrieving climate data...


Data retrieval:   0%|          | 0/1 [00:00<?, ?dataset/s]

You have chosen to produce a time-based Standard Year climate profile centered around 2016 and using scenario SSP 3-7.0. 
Standard year functionality for time-based profiles identifies the closest warming level at that centered year for either 
the input SSP scenario or default 'SSP 3-7.0' if no scenario input is provided. 
The corresponding global warming level for input centered year 2016 will now be determined and used to produce the profile.

Corresponding warming level for 'centered_year'=2016 is [1.12]. 
Now producing the Standard Year climate profile at this warming level.
Using default 'q': 0.5
⚙️  Computing climate profiles...
      📊 Processing 87,600 hours (10 years) of data
      🎯 Computing 50th percentile for each hour of year
      ⚙️ Computing quantiles for 1 warming level(s) and 8 simulation(s)


      Computing profiles:   0%|          | 0/8 [00:00<?, ?combo/s]

KeyboardInterrupt: 

In [7]:
export_profile_to_csv(profile, **profile_selections)

ValueError: Profile MultiIndex should have two or three levels. Found 1 levels.

How...I am so confused. Do SSPs correspond to certain models?

In [17]:
# retrieve_params = {
#     "variable": variable,
#     "resolution": "3 km",
#     # "q": qtile,
#     #"warming_level": [1.13],
#     "units": units,
#     # "no_delta": False,
#     # # approach
#     "approach": "Time",
#     "centered_year": 2016,
#     "time_profile_scenario": "SSP 3-7.0",
#     # # warming level window
#     "warming_level_window": 5,
#     # # Location options -- uncomment based on your desired location type
#     # "stations": station_name,  # uncomment for a weather station
#     # "latitude": (
#     #     latitude - 0.02,
#     #     latitude + 0.02,
#     # ),  # uncomment for a using a custom coordinate location
#     # "longitude": (
#     #     longitude - 0.02,
#     #     longitude + 0.02,
#     # ),  # uncomment for a custom coordinate location
#     "cached_area": area_name,  # uncomment for a cached area
# }

retrieve_params = {
    'variable': 'Air Temperature at 2m',
    'resolution': '9 km',
    # "warming_level": [1.13],
    'units': 'degF',
    'no_delta': True,
    # approach
    "approach": "Time",
    "centered_year": 2016,
    'time_profile_scenario': 'SSP 2-4.5',
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name, # uncomment for a cached area
}

In [18]:
historic_data, future_data = retrieve_profile_data(**retrieve_params)

You have chosen to produce a time-based Standard Year climate profile centered around 2016 and using scenario SSP 2-4.5. 
Standard year functionality for time-based profiles identifies the closest warming level at that centered year for either 
the input SSP scenario or default 'SSP 3-7.0' if no scenario input is provided. 
The corresponding global warming level for input centered year 2016 will now be determined and used to produce the profile.

Corresponding warming level for 'centered_year'=2016 is [1.13]. 
Now producing the Standard Year climate profile at this warming level.


In [20]:
# 2-4.5

future_data

<xarray.DataArray np.str_('Air Temperature at 2m') (warming_level: 1,
                                                    time_delta: 87600,
                                                    simulation: 1)> Size: 350kB
dask.array<getitem, shape=(1, 87600, 1), dtype=float32, chunksize=(1, 1273, 1), chunktype=numpy.ndarray>
Coordinates:
  * warming_level      (warming_level) float64 8B 1.13
    Lambert_Conformal  int64 8B 0
  * time_delta         (time_delta) float64 701kB -4.38e+04 ... 4.38e+04
    centered_year      (simulation) int64 8B 2015
  * simulation         (simulation) <U44 176B 'WRF_CESM2_r11i1p1f1_historical...
Attributes:
    variable_id:           t2
    extended_description:  Temperature of the air 2m above Earth's surface. T...
    units:                 degF
    data_type:             Gridded
    resolution:            9 km
    frequency:             hourly
    location_subset:       ['Los Angeles County']
    approach:              Warming Level
    downscaling_method:    Dynamical
    warming_level_window:  +/- 5 years from centered year
    institution:           UCLA
    grid_mapping:          Lambert_Conformal

In [ ]:
#3-7.0  

future_data

<xarray.DataArray np.str_('Air Temperature at 2m') (warming_level: 1,
                                                    time_delta: 87600,
                                                    simulation: 8)> Size: 3MB
dask.array<getitem, shape=(1, 87600, 8), dtype=float32, chunksize=(1, 2536, 1), chunktype=numpy.ndarray>
Coordinates:
  * warming_level      (warming_level) float64 8B 1.12
    Lambert_Conformal  int64 8B 0
  * time_delta         (time_delta) float64 701kB -4.38e+04 ... 4.38e+04
    centered_year      (simulation) int64 64B 2014 2020 2009 ... 2027 2008 2020
  * simulation         (simulation) <U44 1kB 'WRF_CESM2_r11i1p1f1_historical+...
Attributes:
    variable_id:           t2
    extended_description:  Temperature of the air 2m above Earth's surface. T...
    units:                 degF
    data_type:             Gridded
    resolution:            3 km
    frequency:             hourly
    location_subset:       ['coordinate selection']
    approach:              Warming Level
    downscaling_method:    Dynamical
    warming_level_window:  +/- 5 years from centered year
    institution:           UCLA
    grid_mapping:          Lambert_Conformal